# Analysis of Stock Market Investment Strategies

Author: Charles Lee

This Notebook aims to compare three different investment strategies using historical stock market data.
1. **Timing the Market (Ideal)** - Buying shares when shares are lowest every year
2. **Take Action (Now)** - Buying shares with all their money at the beginning of every year
3. **Dollar Cost Averaging (Average)** - Dividing up the money to buy shares at the beginning of every month

We will also take a look at "worst and best case scenarios".

4. **Unlucky (Worst)** - Saved up all their money and bought all right **before** the 2008 Stock Market Crash and 2020 COVID-19 Pandemic
5. **Omniscient Being (Perfect)** - Saved up all their money and bought all right **after** the 2008 Stock Market Crash and 2020 COVID-19 Pandemic

Finally, we will see how all of these strategies fare against a random investor.

6. **Newbie (Random)** - Buys random amounts of stocks at random times.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
import calendar

## Case Study of VTI
VTI is a highly popular ETF that holds shares from the top companies in America

In [ ]:
vti = pd.read_csv('vti.csv').sort_values('Date').reset_index(drop=True)
vti

In [ ]:
vti_div = pd.read_csv('VTI_Dividends.csv').sort_values('Date').reset_index(drop=True)
vti_div

In [ ]:
standardize_date = lambda string: datetime.datetime.strptime(string, '%Y-%m-%d')
vti['Date'] = vti['Date'].apply(standardize_date)
vti_div['Date'] = vti_div['Date'].apply(standardize_date)

### Here is a graph of VTI's stock price per day since 2001

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='High', data=vti, ci=None)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Stock Price')
g.set_title('VTI High Stock Price Per Day')
sns.despine()

In [ ]:
class Investor():
    def __init__(self, stock, stock_div):
        # Creates portfolio and binds a stock
        self.portfolio = pd.DataFrame(columns=['Date', 'Purchase Price'])
        #self.portfolio_low = pd.DataFrame(columns=['Date', 'Purchase Price'])
        self.salary = 6000
        self.cash = 0
        self.stock = stock
        self.stock_div = stock_div
        self.profits = pd.DataFrame(columns=['Date', 'Profits'])
        self.first_date = self.stock['Date'][0]
        self.last_date = self.stock['Date'][len(vti)-1]
        self.dividends = pd.DataFrame(columns=['Date', 'Dividend Disbursement', 'Total Dividends'])
        
    def income(self):
        # Yearly disbursement of money
        self.cash += self.salary
    
    def create_date(self, string):
        # Transforms date string to datetime object
        if isinstance(string, str):
            return standardize_date(string)
        else:
            return string
    
    def get_nearest_date(self, date, reverse = False):
        # Gets the next closest date of the stock market being open and returns the row of data
        date = self.create_date(date)
        if reverse:
            move = -1
        else:
            move = 1
        # Changes date until found data, unless past first or last date
        while len(self.stock[self.stock['Date'] == date]) < 1:
            if (move == 1 and date > self.last_date):
                # Later than last, return last
                date = self.stock['Date'][len(self.stock)-1]
                row = self.stock[self.stock['Date'] == date]
                return row, date
            elif (move == -1 and date < self.first_date):
                # Earlier than first, return first
                date = self.stock['Date'][0]
                row = self.stock[self.stock['Date'] == date]
                return row, date
            date += datetime.timedelta(days=move)
        # Gets row data from found date
        row = self.stock[self.stock['Date'] == date]
        return row, date
    
    def get_stock_price(self, date):
        date = self.create_date(date)
        row, date = self.get_nearest_date(date)
        return row['High'].values[0]
    
    def buy_stock(self, date):
        # Buys a stock and return True if successful and False if not
        date = self.create_date(date)
        row, date = self.get_nearest_date(date)
        
        # Get stock price
        price = row['High'].values[0]
        
        # Buy if can afford
        if self.cash >= price:
            self.portfolio = self.portfolio.append(pd.DataFrame([[date, price]], columns=['Date', 'Purchase Price'])).reset_index(drop=True)
            self.cash -= price
            return True
        return False
    
    def get_invested(self, date=None):
        # Returns how much money you've put into the stock market
        if date is not None:
            date = self.create_date(date)
            row, date = self.get_nearest_date(date)
            partial = self.portfolio[self.portfolio['Date'] <= date]
            return partial.sum()[0]
        return self.portfolio.sum()[0]
    
    def get_value(self, date=None):
        # Returns how much the stocks are worth at a given date
        if date is not None:
            date = self.create_date(date)
            row, date = self.get_nearest_date(date)
            price = row['High'].values[0]
            partial = self.portfolio[self.portfolio['Date'] <= date]
            return len(partial) * price
        price = self.stock['High'][len(self.stock)-1]
        return len(self.portfolio) * price
    
    def find_dividends(self, date=None):
        # Get portfolio up to date
        div_prof = 0
        for index, row in self.stock_div.iterrows():
            if date is not None and row['Date'] > date:
                break
            div = len(self.portfolio[self.portfolio['Date'] <= row['Date']])*row['Dividends']
            div_prof += div
            self.dividends = self.dividends.append(pd.DataFrame([[row['Date'], div, div_prof]], columns=['Date', 'Dividend Disbursement', 'Total Dividends']))
        self.dividends = self.dividends.reset_index(drop=True)
    
    def find_profits(self, year):
        for i in range(1, 13):
            date = datetime.datetime(year=year, month=i, day=calendar.monthrange(year, i)[1])
            row, date = self.get_nearest_date(date, reverse=True)
            profit = self.get_value(date) - self.get_invested(date)
            self.profits = self.profits.append(pd.DataFrame([[date, profit]], columns=['Date', 'Profits']))
        self.profits = self.profits.drop_duplicates().reset_index(drop=True)
    
    def normalize_data(self):
        #portfolio purchase price
        #profits profits
        #dividends
        four = lambda x: round(x, 4)
        self.portfolio['Purchase Price'] = self.portfolio['Purchase Price'].apply(four)
        self.profits['Profits'] = self.profits['Profits'].apply(four)
        self.dividends['Dividend Disbursement'] = self.dividends['Dividend Disbursement'].apply(four)
        self.dividends['Total Dividends'] = self.dividends['Total Dividends'].apply(four)
    
    def get_lowest(self, year):
        stock_year = self.stock[self.stock['Date'].dt.year == year]
        return stock_year[stock_year['High'] == stock_year['High'].min()]
    
    def execute_strategy(self):
        raise NotImplementedError()

class Ideal(Investor):
    # Buy at the lowest point of every year
    def execute_strategy(self, start, end):
        for i in range(start, end+1):
            self.income()
            lowest = self.get_lowest(i)
            date = lowest['Date'].values[0]
            can_buy = True
            while can_buy:
                can_buy = self.buy_stock(date)
            self.find_profits(i)
        self.find_dividends()
        self.normalize_data()

class Now(Investor):
    # Use all money to buy shares at beginning of year
    def execute_strategy(self, start, end):
        for i in range(start, end+1):
            self.income()
            date = datetime.datetime(year=i, month=1, day=1)
            can_buy = True
            while can_buy:
                can_buy = self.buy_stock(date)
            self.find_profits(i)
        self.find_dividends()
        self.normalize_data()

class Average(Investor):
    # Portion out money over year and buy shares at beginning of each month
    def execute_strategy(self, start, end):
        budget = 0
        for i in range(start, end+1):
            # Every year
            self.income()
            for j in range(1, 13):
                # Every month
                budget += self.salary/12
                date = datetime.datetime(year=i, month=j, day=1)
                try:
                    price = self.get_stock_price(date)
                except:
                    break
                while budget >= price:
                    bought = self.buy_stock(date)
                    if bought:
                        budget -= price
            self.find_profits(i)
        self.find_dividends()
        self.normalize_data()
                    
class Random(Investor):
    # Buys a random number of shares at random times throughout the year
    def execute_strategy(self, start, end):
        for i in range(start, end+1):
            self.income()
            can_buy = True
            while can_buy:
                date = self.get_random_stock(i)['Date'].values[0]
                can_buy = self.buy_stock(date)
            self.find_profits(i)
        self.portfolio = self.portfolio.sort_values('Date').reset_index(drop=True)
        self.find_dividends()
        self.normalize_data()
    
    def get_random_stock(self, year):
        stock_year = self.stock[self.stock['Date'].dt.year == year]
        return stock_year.sample()

# Newbie (Random) Type Investor
This investor spends all of their income every year to buy a random number of shares at random times throughout the year

In [ ]:
rand = Random(vti, vti_div)
rand.execute_strategy(2001, 2021)

This is the leftover uninvested cash by the end of strategy execution

In [ ]:
rand.cash

This is the investor's portfolio by the end of the strategy execution

In [ ]:
rand.portfolio

This is how much money the investor spent on buying the stocks

In [ ]:
rand.get_invested()

This is how much money the investments are worth if we are to sell at the given date

In [ ]:
date = '2021-1-04'
rand.get_value(date)

In [ ]:
rand.profits

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Profits', data=rand.profits)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Profit')
g.set_title('Profits Made Per Month From Investments Using Random strategy')
sns.despine()

This shows the money made from dividends

In [ ]:
rand.dividends

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Total Dividends', data=rand.dividends)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Dividends')
g.set_title('Profits Made Per Month From Dividends Using Random strategy')
sns.despine()

Here, we overlay the profits made from investments and dividends

In [ ]:
rand_merged = rand.profits.merge(rand.dividends, how='outer', on='Date').sort_values('Date').reset_index(drop=True)
rand_merged = rand_merged.fillna(method='pad')
rand_merged['Total Profits'] = rand_merged['Profits'] + rand_merged['Total Dividends']
rand_merged

In [ ]:
x = rand_merged['Date']
y1 = rand_merged['Profits']
y2 = rand_merged['Total Dividends']

y = np.vstack([y1, y2])

labels = ["Profits", "Total Dividends"]

fig, ax = plt.subplots()
ax.stackplot(x, y, labels=labels)
ax.legend(loc='upper left')
plt.title('Total Profits Made Per Month Using Random strategy')
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
ax.set_xticklabels((list(range(2001, 2023))))
ax.set_xlabel('Year')
ax.set_ylabel('Total Profits')
sns.despine()
plt.show()

## Timing the Market (Ideal) Type Investor
This investor uses all their money to buy shares when shares are lowest every year

In [ ]:
ideal = Ideal(vti, vti_div)
ideal.execute_strategy(2001, 2021)

This is the leftover uninvested cash by the end of strategy execution

In [ ]:
ideal.cash

This is the investor's portfolio by the end of the strategy execution

In [ ]:
ideal.portfolio

This is how much money the investor spent on buying the stocks

In [ ]:
ideal.get_invested()

This is how much money the investments are worth if we are to sell at the given date

In [ ]:
date = '2021-1-04'
ideal.get_value(date)

In [ ]:
ideal.profits

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Profits', data=ideal.profits)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Profit')
g.set_title('Profits Made Per Month From Investments Using Ideal strategy')
sns.despine()

This shows the money made from dividends

In [ ]:
ideal.dividends

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Total Dividends', data=ideal.dividends)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Dividends')
g.set_title('Profits Made Per Month From Dividends Using Ideal strategy')
sns.despine()

Here, we overlay the profits made from investments and dividends

In [ ]:
ideal_merged = ideal.profits.merge(ideal.dividends, how='outer', on='Date').sort_values('Date').reset_index(drop=True)
ideal_merged = ideal_merged.fillna(method='pad')
ideal_merged['Total Profits'] = ideal_merged['Profits'] + ideal_merged['Total Dividends']
ideal_merged

In [ ]:
x = ideal_merged['Date']
y1 = ideal_merged['Profits']
y2 = ideal_merged['Total Dividends']

y = np.vstack([y1, y2])

labels = ["Profits", "Total Dividends"]

fig, ax = plt.subplots()
ax.stackplot(x, y, labels=labels)
ax.legend(loc='upper left')
plt.title('Total Profits Made Per Month Using Ideal strategy')
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
ax.set_xticklabels((list(range(2001, 2023))))
ax.set_xlabel('Year')
ax.set_ylabel('Total Profits')
sns.despine()
plt.show()

## Take Action (Now) Type Investor
This investor uses all of their money to buy shares at beginning of year

In [ ]:
now = Now(vti, vti_div)
now.execute_strategy(2001, 2021)

This is the leftover uninvested cash by the end of strategy execution

In [ ]:
now.cash

This is the investor's portfolio by the end of the strategy execution

In [ ]:
now.portfolio

This is how much money the investor spent on buying the stocks

In [ ]:
now.get_invested()

This is how much money the investments are worth if we are to sell at the given date

In [ ]:
date = '2021-1-04'
now.get_value(date)

In [ ]:
now.profits

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Profits', data=now.profits)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Profit')
g.set_title('Profits Made Per Month From Investments Using Now strategy')
sns.despine()

This shows the money made from dividends

In [ ]:
now.dividends

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Total Dividends', data=now.dividends)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Dividends')
g.set_title('Profits Made Per Month From Dividends Using Now strategy')
sns.despine()

Here, we overlay the profits made from investments and dividends

In [ ]:
now_merged = now.profits.merge(now.dividends, how='outer', on='Date').sort_values('Date').reset_index(drop=True)
now_merged = now_merged.fillna(method='pad')
now_merged['Total Profits'] = now_merged['Profits'] + now_merged['Total Dividends']
now_merged

In [ ]:
x = now_merged['Date']
y1 = now_merged['Profits']
y2 = now_merged['Total Dividends']

y = np.vstack([y1, y2])

labels = ["Profits", "Total Dividends"]

fig, ax = plt.subplots()
ax.stackplot(x, y, labels=labels)
ax.legend(loc='upper left')
plt.title('Total Profits Made Per Month Using Now strategy')
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
ax.set_xticklabels((list(range(2001, 2023))))
ax.set_xlabel('Year')
ax.set_ylabel('Total Profits')
sns.despine()
plt.show()

## Dollar Cost Averaging (Average) Type Investor
This investor portions out their money over year and buys shares at beginning of each month (Dollar Cost Averaging)

In [ ]:
avg = Average(vti, vti_div)
avg.execute_strategy(2001, 2021)

This is the leftover uninvested cash by the end of strategy execution

In [ ]:
avg.cash

This is the investor's portfolio by the end of the strategy execution

In [ ]:
avg.portfolio

This is how much money the investor spent on buying the stocks

In [ ]:
avg.get_invested()

This is how much money the investments are worth if we are to sell at the given date

In [ ]:
avg.get_value()

In [ ]:
avg.profits

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Profits', data=avg.profits)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Profit')
g.set_title('Profits Made Per Month From Investments')
sns.despine()

This shows the money made from dividends

In [ ]:
avg.dividends

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Total Dividends', data=avg.dividends)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Dividends')
g.set_title('Profits Made Per Month From Dividends Using Average Strategy')
sns.despine()

Here, we overlay the profits made from investments and dividends

In [ ]:
avg_merged = avg.profits.merge(avg.dividends, how='outer', on='Date').sort_values('Date').reset_index(drop=True)
avg_merged = avg_merged.fillna(method='pad')
avg_merged['Total Profits'] = avg_merged['Profits'] + avg_merged['Total Dividends']
avg_merged

In [ ]:
x = avg_merged['Date']
y1 = avg_merged['Profits']
y2 = avg_merged['Total Dividends']

y = np.vstack([y1, y2])

labels = ["Profits", "Total Dividends"]

fig, ax = plt.subplots()
ax.stackplot(x, y, labels=labels)
ax.legend(loc='upper left')
plt.title('Total Profits Made Per Month Using Average Strategy')
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
ax.set_xticklabels((list(range(2001, 2023))))
ax.set_xlabel('Year')
ax.set_ylabel('Total Profits')
sns.despine()
plt.show()

## Putting it all together

In [ ]:
g_now = now.profits
g_avg = avg.profits
g_ideal = ideal.profits
g_rand = rand.profits

g_now['Strategy'] = 'Now'
g_avg['Strategy'] = 'Average'
g_ideal['Strategy'] = 'Ideal'
g_rand['Strategy'] = 'Random'

In [ ]:
g_all = pd.concat([g_now, g_avg, g_ideal, g_rand]).sort_values('Date').reset_index(drop=True)

In [ ]:
sns.lineplot(data=g_all, x='Date', y='Profits', hue='Strategy')

In [ ]:
date = g_all['Date'][len(g_all)-1]
sns.catplot(x='Strategy', y='Profits', data=g_all[g_all['Date'] == date], kind='bar')

In [ ]:
no_ideal = g_all[g_all['Date'] == date]
no_ideal = no_ideal[no_ideal['Strategy'] != 'Ideal']
g = sns.catplot(x='Strategy', y='Profits', data=no_ideal, kind='bar')
g.set(ylim=(0, 250000))

Factoring in Dividends for Total Profits

In [ ]:
now_merged['Strategy'] = 'Now'
avg_merged['Strategy'] = 'Average'
ideal_merged['Strategy'] = 'Ideal'
rand_merged['Strategy'] = 'Random'

In [ ]:
all_merged = pd.concat([now_merged, avg_merged, ideal_merged, rand_merged]).sort_values('Date').reset_index(drop=True)

In [ ]:
plt.figure(figsize=(16, 6))
sns.lineplot(data=all_merged, x='Date', y='Total Profits', hue='Strategy')

## Unit Tests

In [ ]:
vti = pd.read_csv('test/VTI.csv').sort_values('Date').reset_index(drop=True)
vti_div = pd.read_csv('test/VTI_Dividends.csv').sort_values('Date').reset_index(drop=True)
standardize_date = lambda string: datetime.datetime.strptime(string, '%Y-%m-%d')
vti['Date'] = vti['Date'].apply(standardize_date)
vti_div['Date'] = vti_div['Date'].apply(standardize_date)

In [ ]:
test = Investor(vti, vti_div)
assert test.cash == 0
test.income()
assert test.cash == test.salary

In [ ]:
assert test.create_date('2001-05-31') == datetime.datetime(year=2001, month=5, day=31)

In [ ]:
# Test before first forward
test_row, test_date = test.get_nearest_date('2001-06-13')
assert test_date == test.create_date('2001-06-15')
# Test after last forward
test_row, test_date = test.get_nearest_date('2021-02-17')
assert test_date == test.create_date('2021-02-16')
# Test before first reverse
test_row, test_date = test.get_nearest_date('2001-06-13', True)
assert test_date == test.create_date('2001-06-15')
# Test after last reverse
test_row, test_date = test.get_nearest_date('2021-02-17', True)
assert test_date == test.create_date('2021-02-16')
# Test forward
test_row, test_date = test.get_nearest_date('2021-02-13')
assert test_date == test.create_date('2021-02-16')
# Test reverse
test_row, test_date = test.get_nearest_date('2001-06-17', True)
assert test_date == test.create_date('2001-06-15')

In [ ]:
assert test.get_stock_price('2001-06-15') == 56.005001
assert test.get_stock_price('2001-06-14') == 56.005001
assert test.get_stock_price('2001-06-17') == 55.915001

In [ ]:
# Not enough money
test.cash = 0
assert not test.buy_stock('2001-06-15')
# Enough money
test.cash = 200
assert test.buy_stock('2001-06-15')

In [ ]:
test = Investor(vti, vti_div)
test.cash = 9999999
for i in range(100):
    test.buy_stock('2001-06-15')
    test.buy_stock('2021-02-16')

# All
np.testing.assert_almost_equal(test.get_invested(), 56.005001*100 + 208.389999*100)
# Before specific date
np.testing.assert_almost_equal(test.get_invested('2002-01-01'), 56.005001*100)

In [ ]:
test = Investor(vti, vti_div)
test.cash = 9999999
for i in range(100):
    test.buy_stock('2001-06-15')
    test.buy_stock('2021-02-16')

# All
np.testing.assert_almost_equal(test.get_value(), 200*208.389999)
# Before specific date
np.testing.assert_almost_equal(test.get_value('2001-06-21'), 56.724998*100)

In [ ]:
test = Investor(vti, vti_div)
test.cash = 9999999
for i in range(100):
    test.buy_stock('2001-06-15')

test.find_profits(2001)
np.testing.assert_almost_equal(test.profits[test.profits['Date'] == '2001-06-15']['Profits'].values[0], 0)

np.testing.assert_almost_equal(test.profits[test.profits['Date'] == '2001-06-29']['Profits'].values[0], (56.81-56.005)*100)

Find dividends

In [ ]:
# 10 stocks same date

test = Investor(vti, vti_div)
test.cash = 999999

for i in range(10):
    test.buy_stock('2001-06-15')
test.find_dividends()

assert test.dividends['Dividend Disbursement'][0] == 10*vti_div['Dividends'][0]
assert test.dividends['Dividend Disbursement'][1] == 10*vti_div['Dividends'][1]
assert test.dividends['Total Dividends'][1] == test.dividends['Dividend Disbursement'][0] + test.dividends['Dividend Disbursement'][1]
assert test.dividends['Dividend Disbursement'][len(test.dividends)-1] == 10*vti_div['Dividends'][len(vti_div)-1]

In [ ]:
# 10 stocks each different dates
test = Investor(vti, vti_div)
test.cash = 999999

for i in range(10):
    test.buy_stock('2001-06-15')
    test.buy_stock('2001-09-21')
test.find_dividends()
test.dividends

assert test.dividends['Dividend Disbursement'][0] == 10*vti_div['Dividends'][0]
assert test.dividends['Dividend Disbursement'][1] == 20*vti_div['Dividends'][1]
assert test.dividends['Total Dividends'][1] == test.dividends['Dividend Disbursement'][0] + test.dividends['Dividend Disbursement'][1]
assert test.dividends['Dividend Disbursement'][len(test.dividends)-1] == 20*vti_div['Dividends'][len(vti_div)-1]